# XGBoost regression (multi-node with Dask)

<table>
    <tr>
        <td>
            <img src="https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/dask.png" width="425">
        </td>
        <td>
            <img src="https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/xgboost.png" width="300">
        </td>
    </tr>
</table>

This notebook describes a machine learning training workflow using the famous [NYC Taxi Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). That dataset contains information on taxi trips in New York City.

In this exercise, you'll load data into a [Dask DataFrame](https://docs.dask.org/en/latest/dataframe.html) and use [XGBoost](https://xgboost.readthedocs.io/en/latest/index.html) to answer this question

> based on characteristics that can be known at the beginning of a trip, what tip will this trip earn (as a % of the total fare)?

This notebook gives an introductory tutorial on how to use Dask to scale training of XGBoost models. For more detailed information, see ["Distributed XGBoost with Dask"](https://xgboost.readthedocs.io/en/latest/tutorials/dask.html) in the XGBoost documentation and ["XGBoost Training with Dask"](https://www.saturncloud.io/docs/tutorials/xgboost/) in Saturn Cloud's documentation.

<hr>

## Initialize A Dask Cluster

This tutorial uses multiple machines to show how to apply more computing resources to machine learning training. This is done with Dask. Saturn Cloud offers managed Dask clusters, which can be provisioned and modified programmatically.

The code below creates a Dask cluster using [`dask-saturn`](https://github.com/saturncloud/dask-saturn), the official Dask client for Saturn Cloud. It creates a cluster with the following specs:

* `n_workers=3` --> 3 machines in the cluster
* `scheduler_size='medium'` --> the Dask scheduler will have 4GB of RAM and 2 CPU cores
* `worker_size='large'` --> each worker machine will have 2 CPU cores and 16GB of RAM

To see a list of possible sizes, run the code below.

In [1]:
#from dask.distributed import Client, wait
#from dask_saturn import SaturnCluster

#n_workers = 3
#cluster = SaturnCluster(n_workers=n_workers, scheduler_size="medium", worker_size="large")
#client = Client(cluster)

from dask.distributed import Client, LocalCluster, wait
cluster = LocalCluster()
client = Client(cluster)

/home/test/miniconda3/envs/saturn/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36362 instead
  http_address["port"], self.http_server.port


If you created your cluster here in this notebook, it might take a few minutes for all your nodes to become available. You can run the chunk below to block until all nodes are ready.

>**Pro tip**: Create and/or start your cluster in the Saturn UI if you want to get a head start!

In [2]:
client.wait_for_workers() #n_workers=n_workers)

### Monitor Resource Usage

This tutorial aims to teach you how to take advantage of multiple machines for data science workflows. To prove to yourself that Dask is taking advantage of the resources in the cluster, it's important to understand how to monitor that utilization while your code is running.

Print the `cluster` object in a notebook renders a widget that shows the number of workers, available CPU and memory, and a dashboard link.

In [3]:
cluster

Click that dashboard link to view some diagnostic information about the Dask cluster. This can be used to view the current resource utilization of workers in the cluster and lots of information about what they're currently working on.

<hr>

## Load data

This example is designed to run quickly with small, relatively inexpensive resources. So let's just load a single month of taxi data for training.

In [4]:
import dask.dataframe as dd

taxi = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    assume_missing=True,
).sample(frac=0.3, replace=False)

The code below computes the size of this dataset in memory.

In [5]:
print(f"Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum().compute() / 1e6} MB")

Num rows: 2300337, Size: 464.668074 MB


You can examine the structure of the data with Dask DataFrame commands:

`.head()` = view the first few rows

In [6]:
taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
61859,2.0,2019-01-01 05:54:52,2019-01-01 06:00:36,5.0,2.45,1.0,N,79.0,140.0,2.0,8.5,0.5,0.5,0.00,0.0,0.3,9.80,NaN
28148,2.0,2019-01-01 01:29:24,2019-01-01 01:34:38,1.0,1.17,1.0,N,262.0,141.0,1.0,6.0,0.5,0.5,1.46,0.0,0.3,8.76,NaN
639926,1.0,2019-01-04 07:26:28,2019-01-04 07:29:29,1.0,0.90,1.0,N,48.0,142.0,2.0,5.0,0.0,0.5,0.00,0.0,0.3,5.80,NaN
679907,1.0,2019-01-04 11:10:27,2019-01-04 11:27:38,1.0,2.50,1.0,N,186.0,231.0,1.0,13.0,0.0,0.5,2.75,0.0,0.3,16.55,NaN
425636,2.0,2019-01-03 08:25:57,2019-01-03 08:28:49,1.0,0.51,1.0,N,158.0,158.0,1.0,4.0,0.0,0.5,0.96,0.0,0.3,5.76,NaN


`.dtypes` = list all the columns and the type of data in them

In [7]:
taxi.dtypes

VendorID                        float64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                    float64
DOLocationID                    float64
payment_type                    float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

<hr>

## Prep for Training

Before training a model, we need to transform this dataset into a format that's better-suited to the research question. The function below does that with Dask DataFrame operations.

<details><summary>(click here to learn why data scientists do this)</summary>

**Compute the Target**

The raw data don't contain a column that cleanly describes the tip as a percentage of the total fare. So we need to add one!

**Add Features**

Giving a machine learning model a richer description of each training observation improves its ability to describe the relationship between those observations' characteristics and the target. These characteristics are called "features".

For example, instead of giving a model a raw timestamp, it can be valuable to provide multiple derived characteristics like hour of the day and day of the week. It's plausible, for example, that weekend rides might have a different distribution of tips because they tend to be for leisure, where weekday rides might be mostly people travelling for work.

**Remove Unused Features**

If the goal is to produce a model that could predict the tip for a ride, then characteristics that can only be known AFTER the tip have to be excluded. For example, you can't know the dropoff time or the type of payment until a ride has concluded.

Such features should be dropped before training.
    
</details>

In [8]:
def prep_df(df: dd.DataFrame, target_col: str) -> dd.DataFrame:
    """
    Prepare a raw taxi dataframe for training.
        * computes the target ('tip_fraction')
        * adds features
        * removes unused features
    """
    numeric_feat = [
        "pickup_weekday",
        "pickup_weekofyear",
        "pickup_hour",
        "pickup_week_hour",
        "pickup_minute",
        "passenger_count",
    ]
    categorical_feat = [
        "PULocationID",
        "DOLocationID",
    ]
    features = numeric_feat + categorical_feat
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df[target_col] = df.tip_amount / df.fare_amount

    df["pickup_weekday"] = df.tpep_pickup_datetime.dt.weekday
    df["pickup_weekofyear"] = df.tpep_pickup_datetime.dt.isocalendar().week
    df["pickup_hour"] = df.tpep_pickup_datetime.dt.hour
    df["pickup_week_hour"] = (df.pickup_weekday * 24) + df.pickup_hour
    df["pickup_minute"] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [target_col]].astype(float).fillna(-1)

    return df

Run the code below to get a new data frame, `taxi_train`, that can be used directly for model training.

In [9]:
target_col = "tip_fraction"
taxi_train = prep_df(taxi, target_col)

Dask performs computations in a [lazy manner](https://tutorial.dask.org/01x_lazy.html), so we persist the dataframe to perform data loading and feature processing.

In [10]:
%%time
taxi_train = taxi_train.persist()
_ = wait(taxi_train)

taxi_train.head()

CPU times: user 516 ms, sys: 128 ms, total: 644 ms
Wall time: 16.4 s


,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_week_hour,pickup_minute,passenger_count,PULocationID,DOLocationID,tip_fraction
61859,1.0,1.0,5.0,29.0,54.0,5.0,79.0,140.0,0.000000
28148,1.0,1.0,1.0,25.0,29.0,1.0,262.0,141.0,0.243333
639926,4.0,1.0,7.0,103.0,26.0,1.0,48.0,142.0,0.000000
679907,4.0,1.0,11.0,107.0,10.0,1.0,186.0,231.0,0.211538
425636,3.0,1.0,8.0,80.0,25.0,1.0,158.0,158.0,0.240000


Before going further, check the first few rows of the dataset to make sure that the features look reasonable.

Now that the dataframe has been processed, check its size in memory again.

In [11]:
print(
    f"Num rows: {len(taxi_train)}, Size: {taxi_train.memory_usage(deep=True).sum().compute() / 1e9} GB"
)

Num rows: 2297470, Size: 0.1837976 GB


<hr>

## Train a model

This example uses the native Dask integration built into XGBoost. That integration was added in `xgboost` 1.3.0, and should be preferred to [`dask-xgboost`](https://github.com/dask/dask-xgboost).

In [12]:
import xgboost as xgb

Training data for `xgboost.dask` needs to be prepared in a special object called `DaskDMatrix`. This is like the XGBoost `DMatrix` that you might be familiar with, but is backed by Dask's distributed collections (Dask DataFrame and Dask Array).

In [13]:
features = [c for c in taxi_train.columns if c != target_col]

data = taxi_train[features]
label = taxi_train[target_col]

dtrain = xgb.dask.DaskDMatrix(client=client, data=data, label=label)

You can pass any [xgboost parameters](https://xgboost.readthedocs.io/en/latest/parameter.html) to `xgb.dask.train()`. The training process will then start up on all workers that have some of the data in `dtrain`.

In [14]:
%%time
result = xgb.dask.train(
    client=client,
    params={
        "objective": "reg:squarederror",
        "tree_method": "hist",
        "learning_rate": 0.1,
        "max_depth": 5,
    },
    dtrain=dtrain,
    num_boost_round=50,
)

CPU times: user 228 ms, sys: 56 ms, total: 284 ms
Wall time: 5.69 s


<hr>

## Save model

`xgb.dask.train()` produces a regular `xgb.core.Booster` object, the same model object produced by non-Dask training.

In [15]:
booster = result["booster"]
type(booster)

# xgboost.core.Booster

xgboost.core.Booster

Once you've trained a model, save it in a file to use later for scoring or for comparison with other models.

There are several ways to do this, but `cloudpickle` is likely to give you the best experience. It handles some common drawbacks of the built-in `pickle` library.

`cloudpickle` can be used to write a Python object to bytes, and to create a Python object from that binary representation.

In [16]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/xgboost_dask.pkl", "wb") as f:
    cloudpickle.dump(booster, f)

<hr>

## Calculate metrics on test set

Use a different month for test set

In [17]:
taxi_test = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-02.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    assume_missing=True,
).sample(frac=0.01, replace=False)

taxi_test = prep_df(taxi_test, target_col=target_col)

`xgboost.dask.predict()` can be used to create predictions on a Dask collection using an XGBoost model object. Because the model object here is just a regular XGBoost model, using `xgboost.dask` for batch scoring doesn't require that you also perform training on Dask.

This function returns a Dask Array or Dask Series of predictions, depending on the input type.

In [18]:
preds = xgb.dask.predict(client=client, model=booster, data=taxi_test[features])

The metrics functions in `dask_ml` can compute metrics on Dask collections like Dask Array and Dask DataFrame, so you never have to hold all of the test data in memory on the client. These functions intentionally mimic the metrics functions in scikit-learn.

In [19]:
from dask_ml.metrics import mean_squared_error

mean_squared_error(taxi_test[target_col].to_dask_array(), preds.to_dask_array(), squared=False)

0.4522962203922715

In [20]:
client.close()

<hr>

## Next Steps

In this tutorial, you learned how to use Dask and `xgboost` for distributed model training.

If you want to try working with larger datasets, change the code in this notebook to use more data or explore how to programmatically resize your Dask cluster by following the tips in ["Managing Dask Resources for Machine Learning Training"](https://www.saturncloud.io/docs/tips-and-tricks/training-on-dask).

Alternatively, try [this notebook](./dashboard.ipynb) to learn how to use Saturn Cloud to deploy an interactive dashboard that includes a component which uses the models trained above.

<hr>